### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [7]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
groq_api_key=os.getenv("GROQ_API_KEY")

In [8]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025AB454D2E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025AB454D460>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [10]:
result

AIMessage(content='Bonjour, comment allez-vous ? \n', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 26, 'total_tokens': 37, 'completion_time': 0.022, 'prompt_time': 0.002628462, 'queue_time': None, 'total_time': 0.024628461999999997}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-941505c8-2b46-49a7-ac62-13928e0f7ad5-0', usage_metadata={'input_tokens': 26, 'output_tokens': 11, 'total_tokens': 37})

In [11]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment allez-vous ? \n'

In [12]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, comment allez-vous ? \n\n\nYou could also say:\n\n* Salut, ça va ? (more informal) \n\n'

In [13]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [14]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [15]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:'),
 HumanMessage(content='Hello')]

In [16]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'